In [1]:
#Parser version 2.0




In [1]:
import webbrowser, os
import json
import boto3
import io
from io import BytesIO
import sys
from pprint import pprint
from pdf2image import convert_from_path
from paragraphPar import *


def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
    return rows


def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '    
    return text


def get_table_csv_results(file_name):

    with open(file_name, 'rb') as file:
        img_test = file.read()
        bytes_test = bytearray(img_test)
        print('Image loaded', file_name)

    # process using image bytes
    # get the results
    client = boto3.client('textract')
    
    response = client.analyze_document(Document={'Bytes': bytes_test}, FeatureTypes=['TABLES'])

    # Get the text blocks
    blocks=response['Blocks']
    #pprint(blocks)

    blocks_map = {}
    table_blocks = []
    
    
    for block in blocks:
        
        
        #commented
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)

    if len(table_blocks) <= 0:
        return "<b> NO Table FOUND </b>"

    csv = ''
    for index, table in enumerate(table_blocks):
        csv += generate_table_csv(table, blocks_map, index +1)
        csv += '\n\n'

    return csv

def generate_table_csv(table_result, blocks_map, table_index):
    rows = get_rows_columns_map(table_result, blocks_map)

    table_id = 'Table_' + str(table_index)
    
    # get cells.
    csv = 'Table: {0}\n\n'.format(table_id)

    for row_index, cols in rows.items():
        
        for col_index, text in cols.items():
            csv += '{}'.format(text) + ","
        csv += '\n'
        
    csv += '\n\n\n'
    return csv

def main_conv(file_name, i):
    table_csv = get_table_csv_results(file_name)
    #print(table_csv)
    output_file = 'p_{0}_output_{0}.csv'.format(i, file_name)

    # replace content
    with open(output_file, "wt") as fout:
        fout.write(table_csv)

In [2]:
pages = convert_from_path('Afghanistan.pdf', 100)
i = 0
for page in pages:
    # temporal file
    page.save('temporalA.jpg', 'JPEG') 
    # gives the name:
    main_conv("temporalA.jpg", i)
    i += 1
    
    #page.save('out_{0}.jpg'.format(page), 'JPEG')

Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg
Image loaded temporalA.jpg


KeyboardInterrupt: 

In [4]:
s3BucketName = "textract-console-us-east-1-01b8f23e-f817-4017-bb89-393333f1acd4"
documentName = "GAM.pdf"



jobId = startJob(s3BucketName, documentName)
print("Started job with id: {}".format(jobId))
if(isJobComplete(jobId)):
    response = getJobResults(jobId)

#print(response)

# Print detected text
texts = []
for resultPage in response:
    for item in resultPage["Blocks"]:
        #print(item)
        if item["BlockType"] == "LINE":
            print ('\033[94m' +  item["Text"] + '\033[0m')
            texts.append(item["Text"])
            
    break

Started job with id: bf0d6fe3ee6b55a3e7ac8383642ee3e90a86bde42633c76f4d98d5ae76001ee9
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page recieved: 1
UNAIDS
Strategic Information Department (SID)
Terms of Reference
Analysis of automating data collection elements for GAM 2022-2025
- using machine learning for NLP and text analytics for GAM
1. Background and objective
In May 2016, the United Nations General Assembly High-Level Meeting on Ending AIDS mandated
UN Joint programme on HIV and AIDS (UNAIDS) to collect HIV and AIDS data and annually report
back on progress. The reporting covers all the 195 UN member states, using the existing about 75
key indicators. While the declaration of commitments from 2016 expired in 2020, a new declaration
is presented in June 2021 High-Level Meeting on HIV. The new more specific and numerous targets
for 2025 have been set, and for their monitoring UNAIDS needs to set up an efficient monitoring
framework, minimizing (man

In [5]:
texts

['UNAIDS',
 'Strategic Information Department (SID)',
 'Terms of Reference',
 'Analysis of automating data collection elements for GAM 2022-2025',
 '- using machine learning for NLP and text analytics for GAM',
 '1. Background and objective',
 'In May 2016, the United Nations General Assembly High-Level Meeting on Ending AIDS mandated',
 'UN Joint programme on HIV and AIDS (UNAIDS) to collect HIV and AIDS data and annually report',
 'back on progress. The reporting covers all the 195 UN member states, using the existing about 75',
 'key indicators. While the declaration of commitments from 2016 expired in 2020, a new declaration',
 'is presented in June 2021 High-Level Meeting on HIV. The new more specific and numerous targets',
 'for 2025 have been set, and for their monitoring UNAIDS needs to set up an efficient monitoring',
 'framework, minimizing (manual) reporting burden, and maximizing analytical value.',
 'Currently, the Global AIDS Monitoring (GAM) data collection is done throu